### 1. Acquiring Data

In [1]:
# Steps to analyzing data
# 1. Get the data
# 2. Review the data
# 3. Ask questions based on what you found (get as creative as you need to)
# 4. Clean your data

In [2]:
import unicodecsv as csv
from statistics import mean

In [3]:
# long way
# enrollment_data = []
# file = open('enrollments.csv', 'rb')
# data = csv.DictReader(file)
# for i in data:
#     enrollment_data.append(i)
# file.close()

In [4]:
# shorter way
# with open('enrollments.csv', 'rb') as file:
#     data = csv.DictReader(file)
#     enrollment_data = list(data)

In [5]:
# best way
def read_from_csv(filename):
    with open(filename, 'rb') as file:
        data = csv.DictReader(file)
        return list(data)

In [6]:
enrollment_data = read_from_csv('enrollments.csv')
engagement_data = read_from_csv('engagements.csv')
submission_data = read_from_csv('submissions.csv')

### 2. Cleaning Data

In [7]:
from datetime import datetime as dt

def convert_to_date(date_string):
    if date_string == '':
        return None
    else:
        return dt.strptime(date_string, '%Y-%m-%d')
    
def convert_to_int(int_string):
    if int_string == '':
        return None
    else:
        return int(int_string)
    
def convert_to_float(float_string):
    if float_string == '':
        return None
    else:
        return float(float_string)
    
def convert_to_bool(bool_string):
    if bool_string == '':
        return None
    elif bool_string == 'TRUE':
        return True
    elif bool_string == 'FALSE':
        return False

##### Fix data types

In [8]:
for i in enrollment_data:
    i['join_date'] = convert_to_date(i['join_date'])
    i['cancel_date'] = convert_to_date(i['cancel_date'])
    i['days_to_cancel'] = convert_to_int(i['days_to_cancel'])
    i['is_enrolled'] = convert_to_bool(i['is_enrolled'])
    i['is_canceled'] = convert_to_bool(i['is_canceled'])

In [9]:
enrollment_data[0]

{'account_key': '448',
 'status': 'canceled',
 'join_date': datetime.datetime(2017, 11, 10, 0, 0),
 'cancel_date': datetime.datetime(2018, 1, 14, 0, 0),
 'days_to_cancel': 65,
 'is_enrolled': True,
 'is_canceled': True}

In [10]:
for i in engagement_data:
    i['utc_date'] = convert_to_date(i['utc_date'])
    i['num_courses_visited'] = convert_to_int(i['num_courses_visited'])
    i['total_minutes_visited'] = convert_to_float(i['total_minutes_visited'])
    i['lessons_completed'] = convert_to_float(i['lessons_completed'])
    i['projects_completed'] = convert_to_float(i['projects_completed'])

In [11]:
engagement_data[0]

{'acct': '0',
 'utc_date': datetime.datetime(2018, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0.0,
 'projects_completed': 0.0}

In [12]:
for i in submission_data:
    i['creation_date'] = convert_to_date(i['creation_date'])
    i['completion_date'] = convert_to_date(i['completion_date'])

In [13]:
submission_data[0]

{'creation_date': datetime.datetime(2018, 1, 14, 0, 0),
 'completion_date': datetime.datetime(2018, 1, 16, 0, 0),
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

In [14]:
for i in engagement_data:
    i['account_key'] = i['acct']
    del i['acct']

In [15]:
for i in enrollment_data:
    i['is_student'] = i['is_enrolled']
    del i['is_enrolled']

In [16]:
engagement_data[0]

{'utc_date': datetime.datetime(2018, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0.0,
 'projects_completed': 0.0,
 'account_key': '0'}

In [17]:
print(f"Enrollment records: {len(enrollment_data)}")
print(f"Engagement records: {len(engagement_data)}")
print(f"Submissions records: {len(submission_data)}")

def get_unique_records(data, column_name):
    unique_data = set()
    for i in data:
        unique_data.add(i[column_name])
    return unique_data


print()

unique_enrollment_data = get_unique_records(enrollment_data, 'account_key')
unique_engagement_data = get_unique_records(engagement_data, 'account_key')
unique_submission_data = get_unique_records(submission_data, 'account_key')

print(f"Unique enrollment records: {len(unique_enrollment_data)}")
print(f"Unique engagement records: {len(unique_engagement_data)}")
print(f"Unique submissions records: {len(unique_submission_data)}")

Enrollment records: 1640
Engagement records: 136240
Submissions records: 3642

Unique enrollment records: 1302
Unique engagement records: 1237
Unique submissions records: 743


In [18]:
outliers = 0
for i in enrollment_data:
    if i['account_key'] not in unique_engagement_data and i['join_date'] != i['cancel_date']:
        outliers +=1
        print(i, end='\n\n')
print(outliers)

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2018, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2018, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_canceled': True, 'is_student': True}

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2018, 3, 10, 0, 0), 'cancel_date': datetime.datetime(2018, 6, 17, 0, 0), 'days_to_cancel': 99, 'is_canceled': True, 'is_student': True}

{'account_key': '1101', 'status': 'current', 'join_date': datetime.datetime(2018, 2, 25, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_canceled': False, 'is_student': True}

3


In [19]:
# Check for possible test accounts.
# is_student shows True for questionable looking accounts, but shows False for legit accounts
test_accounts = set()
for i in enrollment_data:
    if i['is_student']:
        test_accounts.add(i['account_key'])
print(f"{len(test_accounts)} possible test accounts were found.")

6 possible test accounts were found.


In [20]:
def remove_test_accounts(data):
    new_records = []
    for i in data:
        if i['account_key'] in test_accounts:
            pass
        else:
            new_records.append(i)
    return new_records

In [21]:
true_enrollments = remove_test_accounts(enrollment_data)
true_engagements = remove_test_accounts(engagement_data)
true_submissions = remove_test_accounts(submission_data)

print(f"True enrollment records: {len(true_enrollments)}")
print(f"True engagement records: {len(true_engagements)}")
print(f"True submissions records: {len(true_submissions)}")

True enrollment records: 1622
True engagement records: 135656
True submissions records: 3634


### 3. Answering Questions w/ Data

In [22]:
# What was the average number of days to cancel?
days_to_cancel = mean([i['days_to_cancel'] for i in enrollment_data if i['days_to_cancel'] is not None])
print(f"It took students about {round(days_to_cancel, 1)} days to cancel their subscription.")

It took students about 43.9 days to cancel their subscription.


#### Find all quality accounts where:
##### 1. 'is_canceled' is False - accounts are currently active
##### 2. 'days_to_cancel' is greater than 7 - people who lasted past the free trial
##### 3. retrieve one record with most recent 'join_date', but have account keys that show up multiple times - account holder may have stopped and started their trials many times. Only pull the most recent one, so we don't screw up our data

In [23]:
# Find accounts that are in good standing and fit all above criteria
accounts_in_good_standing = dict()
for i in true_enrollments:
    if not i['is_canceled'] or i['days_to_cancel'] > 7:
        if i['account_key'] not in accounts_in_good_standing or i['join_date'] > accounts_in_good_standing[i['account_key']]:
            accounts_in_good_standing[i['account_key']] = i['join_date']
len(accounts_in_good_standing)

# { 'account_key': datetime_objects }

995

In [24]:
# accounts_in_good_standing

In [25]:
# Remove all free trials
def remove_free_trials(data):
    free_trials_list = []
    for i in data:
        if i['account_key'] in accounts_in_good_standing:
            free_trials_list.append(i)
    return free_trials_list

quality_enrollments = remove_free_trials(true_enrollments)
quality_engagements = remove_free_trials(true_engagements)
quality_submissions = remove_free_trials(true_submissions)

print(f"{len(quality_enrollments)} Quality enrollments")
print(f"{len(quality_engagements)} Quality engagements")
print(f"{len(quality_submissions)} Quality submissions")

1293 Quality enrollments
134549 Quality engagements
3618 Quality submissions


In [26]:
# Question: How many enrollees have active engagement within the first week?
def engagements_within_first_week(date_joined, date_of_first_engagement):
    time_difference = date_of_first_engagement - date_joined
    if time_difference.days >= 0 and time_difference.days < 7:
        return True
    
for i in quality_engagements:
    if i['num_courses_visited'] > 0:
        i['had_visited'] = 1
    else:
        i['had_visited'] = 0
    
first_week_engagements = []
for i in quality_engagements:
    if engagements_within_first_week(accounts_in_good_standing[i['account_key']], i['utc_date']):
        first_week_engagements.append(i)
        
print(f"{len(first_week_engagements)}: of all quality accounts, these are the engagements within the first week.")

6919: of all quality accounts, these are the engagements within the first week.


In [27]:
# first_week_engagements[100]
# { 'account_key': num_minutes_spent }
from collections import defaultdict

def compare(data, key_):
    new_data = defaultdict(list)
    for i in data:
        key = i[key_]
        new_data[key].append(i)
    return new_data

engagements_grouped_by_account = compare(first_week_engagements, 'account_key')
# for i in engagements_grouped_by_account.items():
#     print(i)
#     break

In [28]:
# Question: How many minutes did each student spend online during the first week

def sum_records(data, field):
    new_data = dict()
    for k, v in data.items():
        sum_ = 0
        for i in v:
            sum_ += i[field]
        new_data[k] = sum_
    return new_data
        
minutes_per_account = sum_records(engagements_grouped_by_account, 'total_minutes_visited')
minutes_per_account

{'0': 494.88049616599994,
 '1': 18.576384666670002,
 '2': 0.0,
 '3': 0.0,
 '4': 33.3214046667,
 '5': 329.7578566663,
 '6': 780.4545511666701,
 '7': 104.20388850009999,
 '8': 989.113641833,
 '9': 448.471384167,
 '10': 130.12347833367,
 '11': 0.0,
 '12': 179.4719088333,
 '13': 1013.3833969996999,
 '14': 65.6221875,
 '15': 983.375040335,
 '16': 119.12030049999998,
 '17': 235.49969150033,
 '18': 155.1361575,
 '19': 0.0,
 '20': 447.93897783336996,
 '21': 931.1036911666699,
 '22': 657.2052335000001,
 '23': 1591.3228143334,
 '24': 943.188117167,
 '25': 0.0,
 '26': 430.801675833,
 '27': 1579.12122666663,
 '28': 766.256315667,
 '29': 556.1906033333,
 '30': 69.6578351667,
 '31': 0.0,
 '32': 123.2915048333,
 '33': 253.9870258334,
 '34': 180.413814,
 '35': 765.6402170004,
 '36': 809.2138958339,
 '37': 1378.195091668,
 '38': 0.0,
 '39': 1001.5888595,
 '40': 478.21952616690004,
 '41': 511.925391,
 '42': 576.4643026663,
 '43': 88.822038,
 '44': 0.0,
 '45': 0.0,
 '46': 26.1189351667,
 '47': 854.828888

In [29]:
# Find the account holder who put in the most minutes

courses_visited_per_account = sum_records(engagements_grouped_by_account, 'num_courses_visited')
courses_visited_per_account

for k, v in courses_visited_per_account.items():
    if v == max(courses_visited_per_account.values()):
        print(k, v)

317 25


In [30]:
# for k, v in engagements_grouped_by_account.items():
#     print(k, v)
#     break

In [31]:
# quality_engagements[10]

In [32]:
sum(minutes_per_account.values()) / sum(courses_visited_per_account.values())

77.08380529923245

In [33]:
# What's the ratio of passed/unpassed submissions?b

In [34]:
for i in quality_submissions:
    if i['processing_state'] != 'evaluated'.upper():
        print(i)

{'creation_date': datetime.datetime(2018, 6, 4, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '313', 'lesson_key': '3176718735', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 2, 12, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '474', 'lesson_key': '3562208770', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 27, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '452', 'lesson_key': '3174288624', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 26, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '500', 'lesson_key': '3165188753', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 27, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '559', 'lesson_key': '3165188753', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 26, 0, 0), 'completion_date': N

In [35]:
for i in quality_submissions:
    if i['completion_date'] is None:
        print(i)

{'creation_date': datetime.datetime(2018, 6, 4, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '313', 'lesson_key': '3176718735', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 2, 12, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '474', 'lesson_key': '3562208770', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 27, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '452', 'lesson_key': '3174288624', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 26, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '500', 'lesson_key': '3165188753', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 27, 0, 0), 'completion_date': None, 'assigned_rating': '', 'account_key': '559', 'lesson_key': '3165188753', 'processing_state': 'CREATED'}
{'creation_date': datetime.datetime(2018, 8, 26, 0, 0), 'completion_date': N

### 4. Finalizing/Summarizing Data

In [36]:
# I wrote my conclusion here

### DO-OVER

In [37]:
# install pandas, unmpy and matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
enrollment_df = pd.read_csv('enrollments.csv')
engagement_df = pd.read_csv('engagements.csv')
submission_df = pd.read_csv('submissions.csv')

In [39]:
pd.Timestamp('2017-11-10').year

2017

In [40]:
enrollment_df

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
0,448,canceled,2017-11-10,2018-01-14,65.0,True,True
1,448,canceled,2017-11-05,2017-11-10,5.0,True,True
2,448,canceled,2018-01-27,2018-01-27,0.0,True,True
3,448,canceled,2017-11-10,2017-11-10,0.0,True,True
4,448,current,2018-03-10,NaN,NaN,True,False
...,...,...,...,...,...,...,...
1635,1176,current,2018-08-12,NaN,NaN,False,False
1636,1110,current,2018-08-13,NaN,NaN,False,False
1637,1116,canceled,2018-08-15,2018-08-18,3.0,False,True
1638,874,current,2018-08-22,NaN,NaN,False,False


In [41]:
# help(enrollment_df)
enrollment_df.columns
enrollment_df.index
enrollment_df.dtypes

account_key         int64
status             object
join_date          object
cancel_date        object
days_to_cancel    float64
is_enrolled          bool
is_canceled          bool
dtype: object

In [42]:
d = {
    'my_column': [1, 2, 3], 
    'your_column': [3, 4, 5], 
    'another_column': [5, 6, 7]
}
index = ['first_row', 'second_row', 'third_row', 'fourth_row', 'fifth_row', 'sixth_row', 'seventh_row', 'eighth row']
columns = ['first column', 'second column', 'third column', 'fourth column', 'fifth column', 'sixth column', 'seventh_column', 'eighth_column']
new_d = [[1, 2, "Hello", 4, 5, 6, '9-5', 98], [7, 8, "Goodbye", 10, 11, 12, '10-4', 97], [13, 14, "Greetings", 16, 17, 18, '12-7', np.nan], [19, 20, "Salutations", 22, 23, 24, '7-6', np.nan], [25, 26, "Bonjour", 28, 29, 30, '6-10', np.nan], [31, 32, "Salut", 34, 35, 36, '12-8', 93], [37, 38, "Au Revoir", 40, 41, 42, '9-12', 92], [43, np.nan, "Allo", 46, np.nan, 48, '12-9', 91]]

test_df = pd.DataFrame(data=new_d, index=index, columns=columns)
test_df

,first column,second column,third column,fourth column,fifth column,sixth column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,6,9-5,98.0
second_row,7,8.0,Goodbye,10,11.0,12,10-4,97.0
third_row,13,14.0,Greetings,16,17.0,18,12-7,NaN
fourth_row,19,20.0,Salutations,22,23.0,24,7-6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,30,6-10,NaN
sixth_row,31,32.0,Salut,34,35.0,36,12-8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,42,9-12,92.0
eighth row,43,NaN,Allo,46,NaN,48,12-9,91.0


In [43]:
# pd.DataFrame(d, index=index)

In [44]:
# enrollment_df['days_to_cancel']
# enrollment_df.days_to_cancel

# test_df['tertiary column']
# enrollment_df.tertiary column

# Change a column name
# enrollment_df.replace?

# If you do not add inplace as an argument to some dataframe methods, your object will not be changed.
# Solution: add either inplace, or set new dataframe expression/evaluation to the original dataframe

# test_df = test_df.rename(columns={'my column': 'my_column'})
# test_df.rename(columns={'my column': 'my_column'}, inplace=True)
# test_df

test_df.columns = test_df.columns.str.replace(' ', '_')
test_df

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,6,9-5,98.0
second_row,7,8.0,Goodbye,10,11.0,12,10-4,97.0
third_row,13,14.0,Greetings,16,17.0,18,12-7,NaN
fourth_row,19,20.0,Salutations,22,23.0,24,7-6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,30,6-10,NaN
sixth_row,31,32.0,Salut,34,35.0,36,12-8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,42,9-12,92.0
eighth row,43,NaN,Allo,46,NaN,48,12-9,91.0


In [45]:
# for i in enrollment_data:
#     i['join_date'] = convert_to_date(i['join_date'])
#     i['cancel_date'] = convert_to_date(i['cancel_date'])
#     i['days_to_cancel'] = convert_to_int(i['days_to_cancel'])
#     i['is_enrolled'] = convert_to_bool(i['is_enrolled'])
#     i['is_canceled'] = convert_to_bool(i['is_canceled'])

In [46]:
# Check for number of records you have
# len(enrollment_df)
# enrollment_df.info()
enrollment_df.tail(1)

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
1639,686,current,2018-08-23,NaN,NaN,False,False


In [47]:
# Check for n number of beginning records
# .head(n)
enrollment_df.head(20)

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
0,448,canceled,2017-11-10,2018-01-14,65.0,True,True
1,448,canceled,2017-11-05,2017-11-10,5.0,True,True
2,448,canceled,2018-01-27,2018-01-27,0.0,True,True
3,448,canceled,2017-11-10,2017-11-10,0.0,True,True
4,448,current,2018-03-10,NaN,NaN,True,False
5,448,canceled,2018-01-14,2018-01-27,13.0,True,True
6,448,canceled,2018-01-27,2018-03-10,42.0,True,True
7,448,canceled,2018-01-27,2018-01-27,0.0,True,True
8,448,canceled,2018-01-27,2018-01-27,0.0,True,True
9,700,canceled,2017-11-10,2017-11-16,6.0,False,True


In [48]:
# Check for n number of ending records
# .tail(n)
enrollment_df.tail(15)

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
1625,1057,canceled,2018-07-17,2018-07-21,4.0,False,True
1626,689,current,2018-07-17,NaN,NaN,False,False
1627,1278,current,2018-07-17,NaN,NaN,False,False
1628,1272,canceled,2018-07-17,2018-07-23,6.0,False,True
1629,438,current,2018-07-17,NaN,NaN,False,False
1630,992,current,2018-07-17,NaN,NaN,False,False
1631,575,current,2018-07-17,NaN,NaN,False,False
1632,807,canceled,2018-07-20,2018-07-22,2.0,False,True
1633,1298,current,2018-08-11,NaN,NaN,False,False
1634,751,current,2018-08-11,NaN,NaN,False,False


In [49]:
enrollment_df.head(1)

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
0,448,canceled,2017-11-10,2018-01-14,65.0,True,True


In [50]:
# Index into your DataFrame
# enrollment_df.iloc[0]

# enrollment_df.iloc[0:4, 0:5] # gets certain number of records for certain number of columns
# enrollment_df.iloc[:, 0:5] # gets all records and shows certain number of columns
# enrollment_df.iloc[0:4, :] # gets all data for each record
# enrollment_df.iloc[:, [1, 4, 5, 6]] # if you want to get specific columns, you have to pick them manually

In [51]:
# test_df = pd.DataFrame(d, index=index)
# test_df.loc['first_row']

In [52]:
enrollment_df.loc[1625]

account_key             1057
status              canceled
join_date         2018-07-17
cancel_date       2018-07-21
days_to_cancel             4
is_enrolled            False
is_canceled             True
Name: 1625, dtype: object

In [53]:
test_df.iloc[0]

first_column          1
second_column         2
third_column      Hello
fourth_column         4
fifth_column          5
sixth_column          6
seventh_column      9-5
eighth_column        98
Name: first_row, dtype: object

In [54]:
test_df.loc['second_row':'fifth_row']

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
second_row,7,8.0,Goodbye,10,11.0,12,10-4,97.0
third_row,13,14.0,Greetings,16,17.0,18,12-7,NaN
fourth_row,19,20.0,Salutations,22,23.0,24,7-6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,30,6-10,NaN


In [55]:
# Whenever performing slicing using either .loc or .iloc, neither of the stop values are inherently exclusive
# It's just that, by default, .iloc's indeces start at 0, which gives the illusion that the slices are exclusive

In [56]:
test_df.sixth_column = test_df.sixth_column.apply(lambda x:x**2)

In [57]:
test_df

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,36,9-5,98.0
second_row,7,8.0,Goodbye,10,11.0,144,10-4,97.0
third_row,13,14.0,Greetings,16,17.0,324,12-7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,7-6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,6-10,NaN
sixth_row,31,32.0,Salut,34,35.0,1296,12-8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,9-12,92.0
eighth row,43,NaN,Allo,46,NaN,2304,12-9,91.0


In [58]:
test_df.dtypes

first_column        int64
second_column     float64
third_column       object
fourth_column       int64
fifth_column      float64
sixth_column        int64
seventh_column     object
eighth_column     float64
dtype: object

In [59]:
# test_df.seventh_column.apply(lambda x: x[-1])

In [60]:
d1 = test_df.copy()
d2 = test_df.copy()

d1.seventh_column = d1.seventh_column.str.extract('(\d)')
d2.seventh_column = d2.seventh_column.apply(lambda x: x[-1])

test_df
# test_df.seventh_column.str.replace('-', '/')

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,36,9-5,98.0
second_row,7,8.0,Goodbye,10,11.0,144,10-4,97.0
third_row,13,14.0,Greetings,16,17.0,324,12-7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,7-6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,6-10,NaN
sixth_row,31,32.0,Salut,34,35.0,1296,12-8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,9-12,92.0
eighth row,43,NaN,Allo,46,NaN,2304,12-9,91.0


In [61]:
d1

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,36,9,98.0
second_row,7,8.0,Goodbye,10,11.0,144,1,97.0
third_row,13,14.0,Greetings,16,17.0,324,1,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,7,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,6,NaN
sixth_row,31,32.0,Salut,34,35.0,1296,1,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,9,92.0
eighth row,43,NaN,Allo,46,NaN,2304,1,91.0


In [62]:
d2

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,36,5,98.0
second_row,7,8.0,Goodbye,10,11.0,144,4,97.0
third_row,13,14.0,Greetings,16,17.0,324,7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,0,NaN
sixth_row,31,32.0,Salut,34,35.0,1296,8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,2,92.0
eighth row,43,NaN,Allo,46,NaN,2304,9,91.0


In [63]:
# Drop original records and add the new ones
for i in test_df.index:
    test_df.drop(index=i, inplace=True)
    test_df = test_df.append(d1.loc[i])
    test_df = test_df.append(d2.loc[i])
    
test_df

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,1,2.0,Hello,4,5.0,36,9,98.0
first_row,1,2.0,Hello,4,5.0,36,5,98.0
second_row,7,8.0,Goodbye,10,11.0,144,1,97.0
second_row,7,8.0,Goodbye,10,11.0,144,4,97.0
third_row,13,14.0,Greetings,16,17.0,324,1,NaN
third_row,13,14.0,Greetings,16,17.0,324,7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,0,NaN


In [64]:
# Show column names
for i in test_df:
    print(i)

first_column
second_column
third_column
fourth_column
fifth_column
sixth_column
seventh_column
eighth_column


In [65]:
# Show one record of data

for i in test_df.index:
    print(test_df.loc[i])
    break
#     print(test_df.iloc[i])

           first_column  second_column third_column  fourth_column  \
first_row             1            2.0        Hello              4   
first_row             1            2.0        Hello              4   

           fifth_column  sixth_column seventh_column  eighth_column  
first_row           5.0            36              9           98.0  
first_row           5.0            36              5           98.0  


In [66]:
# Figuring out what to do with missing data
test_df.isna().head()

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
first_row,False,False,False,False,False,False,False,False
first_row,False,False,False,False,False,False,False,False
second_row,False,False,False,False,False,False,False,False
second_row,False,False,False,False,False,False,False,False
third_row,False,False,False,False,False,False,False,True


In [67]:
test_df.isnull().tail()

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
sixth_row,False,False,False,False,False,False,False,False
seventh_row,False,False,False,False,False,False,False,False
seventh_row,False,False,False,False,False,False,False,False
eighth row,False,True,False,False,True,False,False,False
eighth row,False,True,False,False,True,False,False,False


In [68]:
test_df[test_df['first_column'] > 15]

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
fourth_row,19,20.0,Salutations,22,23.0,576,7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,0,NaN
sixth_row,31,32.0,Salut,34,35.0,1296,1,93.0
sixth_row,31,32.0,Salut,34,35.0,1296,8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,9,92.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,2,92.0
eighth row,43,NaN,Allo,46,NaN,2304,1,91.0
eighth row,43,NaN,Allo,46,NaN,2304,9,91.0


In [69]:
test_df.query('first_column > 15')

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
fourth_row,19,20.0,Salutations,22,23.0,576,7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,6,NaN
fifth_row,25,26.0,Bonjour,28,29.0,900,0,NaN
sixth_row,31,32.0,Salut,34,35.0,1296,1,93.0
sixth_row,31,32.0,Salut,34,35.0,1296,8,93.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,9,92.0
seventh_row,37,38.0,Au Revoir,40,41.0,1764,2,92.0
eighth row,43,NaN,Allo,46,NaN,2304,1,91.0
eighth row,43,NaN,Allo,46,NaN,2304,9,91.0


In [70]:
# test_df.query('third_column == \'Salutations\'')
test_df.query("third_column == 'Salutations'")

,first_column,second_column,third_column,fourth_column,fifth_column,sixth_column,seventh_column,eighth_column
fourth_row,19,20.0,Salutations,22,23.0,576,7,NaN
fourth_row,19,20.0,Salutations,22,23.0,576,6,NaN


### Homework

In [71]:
# What was the average number of days to cancel?
enrollment_df.days_to_cancel.mean()

# How many minutes did each student spend online during the first week


# How many enrollees have active engagement within the first week?

# Find the account holder who put in the most minutes.

43.917004048582996

In [72]:
import time as t
startt = t.time()
print(startt)

1601579682.0778685


In [73]:
starttime = t.time()
starttime
fwe_df = pd.DataFrame(data = [[x, engagement_df.query(f"acct == {x}").head(7)['total_minutes_visited'].sum()] for x in list(set(list(engagement_df['acct'])))] , columns = ['Acct_no', 'First_Week_Engagement'])
print(t.time()-starttime)

3.8897194862365723


In [74]:
fwe_df

,Acct_no,First_Week_Engagement
0,0,494.880496
1,1,18.576385
2,2,0.000000
3,3,827.906774
4,4,33.321405
...,...,...
1232,1300,13.387217
1233,1301,6.202126
1234,1302,0.000000
1235,1303,35.315063


In [75]:
i = 1301
engagement_df.query(f"acct == {i}").head(7)['total_minutes_visited'].sum()

6.2021255

In [76]:
list(set(list(engagement_df['acct'])))[0]

0